In [ ]:
import numpy as np 
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import sklearn as sk

# Step 1: Ticker
TICKER = "SPY"

# Step 2: Price Behavior 
INTERVAL = "1m"
if INTERVAL == "1h":
    PERIOD = "730d"
else:
    PERIOD = "max"

# Step 3: Price difference
SHIFT = 1
    # Tomorrow's Close Price in the same Row as today

# Step 5: Data Subsection
LOOKBACK = 10000

def get_data(ticker = TICKER, interval = INTERVAL, lookback = LOOKBACK):

    # Step 1: Download Data
    df = yf.download(ticker, interval = interval, period = PERIOD)

    # Step 2: Select only Level 0 of Multilevel Columns
    df.columns = df.columns.get_level_values(0)

    # Step 3: Simplifying df for Plots
    df = df.reset_index(drop=True)

    return df.iloc[-lookback:, :]
        # -lookback: all loockback rows
        # : all columns 

# Defining y/the Dependent/the Target Variable
# --> We can create a strategy, that could forecast positively/negatively 
# with our Close Price changes 

# Lets inspect how a tech Indicator is related to how a price changes tomorrow
# For eg. RSI - you want to see a negative correlation between value of the RSI
# and Close Price Change the following day. Because the higher the value of RSI, 
# the more overbought the asset and shoudl decrease soon 
def add_target(df, shift = SHIFT):

    # Step 1:
    df["diff()"] = df["Close"].diff()

    # Step 2: 
    df[f"diff().shift({SHIFT})"] = df["Close"].diff().shift(-SHIFT)
    # diff(): Today's Price - Yesterday's Price == how much money you made or lost today
    # shift(-1): shift backwards in time. It takes the diff value from tomorrow into the row for today
    # We are trying to find a correlation with the price of tomorrow

    # Step 3: 


    return df


def main():
    df = get_data()
    df = add_target(df)

    return df

df = main()
df

/var/folders/jj/9j0c_6196zx24tx5jb3ngnym0000gn/T/ipykernel_11202/1548517460.py:26: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, interval = interval, period = PERIOD)
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume,diff(),diff().shift(1)
0,680.500000,680.520020,679.900024,679.950012,1451696,NaN,-0.145020
1,680.354980,680.510010,680.200012,680.510010,239244,-0.145020,-0.184998
2,680.169983,680.390015,680.140015,680.344971,180502,-0.184998,0.085693
3,680.255676,680.280029,680.130005,680.179993,126456,0.085693,-0.190674
4,680.065002,680.340027,679.919983,680.250000,170149,-0.190674,0.085022
...,...,...,...,...,...,...,...
2335,661.309998,661.679993,661.039978,661.289978,954101,0.020020,-0.309998
2336,661.000000,661.359985,660.940002,661.299988,510035,-0.309998,-0.150024
2337,660.849976,661.109985,660.770020,661.010010,1198968,-0.150024,-0.375000
2338,660.474976,661.034973,660.250000,660.840027,1648967,-0.375000,-0.334961
